In [0]:
# Info
# Challange Name : Jigsaw Multilingual Toxic Comment Classification
# Link : https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification

# Proceeding >>>
# i.   used pretrained BERT (Bidirectional Encoder Representations for Transformers) for toxic text classification task.
# ii.  Finetuned model with additional classification layer for 2 epochs with adam optimizer with lr = .00005, batch_size = 32.
# iii. training accuracy = 95.50%, validation accuracy = 84.69%

# Dependencies
# i.   tensorflow version 1.x, tensorflow_hub, bert-tensorflow
# ii.  keras, numpy, pandas

In [0]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model, load_model
from keras.layers import Dropout, Dense, Embedding, Input, LSTM, Bidirectional, GlobalAveragePooling2D, Layer
from keras import optimizers
from keras.activations import sigmoid
from keras.preprocessing import sequence
import tensorflow_hub as hub

In [0]:
!pip install bert-tensorflow

In [0]:
class BertLayer(Layer):
    def __init__(self, **kwargs):
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module('https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1', trainable=True)

        trainable_vars = self.bert.variables       
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

        for var in trainable_vars:
            self._trainable_weights.append(var)
        
        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["pooled_output"]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.output_size)

In [0]:
# Metadata
train_file = '/content/drive/My Drive/Project_BertoX/train-processed-seqlen128.csv'
validation_file = '/content/drive/My Drive/Project_BertoX/validation-processed-seqlen128.csv'
batch_size = 32
train_len = 223549 # len(pd.read_csv(train_file))
valid_len = 8000   # len(pd.read_csv(validation_file))
learning_rate = 0.00005

In [0]:
# Data Generator
import pandas as pd
import numpy as np

def DataGen(filename, batch = 32):

  df = pd.read_csv(filename)
  df = df[['input_word_ids','input_mask', 'all_segment_id', 'toxic']]
  count = 0
  inp_ids = []
  inp_masks = []
  seg_ids = []
  while (True):

    cols = df.columns
    inp_id = df[cols[0]][count:count+batch].tolist()
    inp_mask = df[cols[1]][count:count+batch].tolist()
    seg_id = df[cols[2]][count:count+batch].tolist()
    labels = df[cols[3]][count:count+batch].tolist()

    # converting string tuple into integer list
    for i in range(batch):
      inp_ids.append([int(x) for x in inp_id[i][1:-1].split(', ')])
      inp_masks.append([int(x) for x in inp_mask[i][1:-1].split(', ')])
      seg_ids.append([int(x) for x in seg_id[i][1:-1].split(', ')])

    count+=batch
    if(count+batch>=len(df)):
      count=0

    yield ([np.array(inp_ids), np.array(inp_masks), np.array(seg_ids)], np.array(labels))
    inp_ids = []
    inp_masks = []
    seg_ids = []

In [0]:
train_gen = DataGen(filename=train_file, batch = batch_size)
val_gen = DataGen(filename=validation_file, batch = batch_size)
x = train_gen.__next__()
print(x[0][0].shape,x[0][1].shape,x[0][2].shape, x[1].shape)

In [0]:
def Bertox(Inp_shape):

    in_id = Input(shape=Inp_shape)
    in_mask = Input(shape=Inp_shape)
    in_segment = Input(shape=Inp_shape)
    bert_inputs = [in_id, in_mask, in_segment]

    bert_output = BertLayer()(bert_inputs)
    do = Dropout(0.1)(bert_output)
    pred = Dense(1, activation='sigmoid')(do)

    model = Model(inputs=bert_inputs, outputs=pred)
    return model

In [0]:
model = Bertox((128,))
model.summary()

In [0]:
Adam = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])
train_record = model.fit_generator(train_gen, steps_per_epoch=train_len/batch_size, validation_data= val_gen,
                                   validation_steps=valid_len/batch_size, epochs=2, shuffle=True, initial_epoch=0)

In [0]:
model.save('/content/drive/My Drive/BertoX.h5')

In [0]:
# Testing
df = pd.read_csv('/content/drive/My Drive/Project_BertoX/test-processed-seqlen128.csv')
df = df[['input_word_ids','input_mask', 'all_segment_id']]
cols = df.columns
inp_ids = []
inp_masks = []
seg_ids = []

cols = df.columns
inp_id = df[cols[0]].tolist()
inp_mask = df[cols[1]].tolist()
seg_id = df[cols[2]].tolist()

for i in range(len(df)):
  inp_ids.append([int(x) for x in inp_id[i][1:-1].split(', ')])
  inp_masks.append([int(x) for x in inp_mask[i][1:-1].split(', ')])
  seg_ids.append([int(x) for x in seg_id[i][1:-1].split(', ')])
print(np.array(inp_ids).shape, np.array(inp_masks).shape, np.array(seg_ids).shape)

prediction = model.predict([np.array(inp_ids), np.array(inp_masks), np.array(seg_ids)])

In [0]:
pred_frame = pd.DataFrame(prediction)
pred_frame.to_csv('predict.csv', index=False)